In [2]:
#libraries
import pandas as pd
import numpy as np
from apyori import apriori


In [3]:
df = pd.read_csv('/Users/harikrishnans/Downloads/Groceries_dataset.csv')

In [4]:
df.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [6]:
#grouping
data = df.groupby(['Member_number', 'Date'])['itemDescription'].apply(lambda x: ', '.join(x)).reset_index()

In [7]:
data.drop(['Member_number', 'Date'], axis=1, inplace=True)
data

,itemDescription
0,"sausage, whole milk, semi-finished bread, yogurt"
1,"whole milk, pastry, salty snack"
2,"canned beer, misc. beverages"
3,"sausage, hygiene articles"
4,"soda, pickled vegetables"
...,...
14958,"tropical fruit, berries, other vegetables, yog..."
14959,"bottled water, herbs"
14960,"fruit/vegetable juice, onions"
14961,"soda, root vegetables, semi-finished bread"


In [8]:
# Save to CSV
data.to_csv("Description.csv", index=False)

In [11]:
# Split itemDescription into separate items
split_data = data['itemDescription'].str.split(',', expand=True)

In [12]:
split_data.isnull().sum()

0         0
1         0
2     10080
3     12778
4     14168
5     14512
6     14687
7     14767
8     14912
9     14962
10    14962
dtype: int64

In [13]:
# Fill NaN values with empty strings
split_data.fillna('', inplace=True)
split_data

,0,1,2,3,4,5,6,7,8,9,10
0,sausage,whole milk,semi-finished bread,yogurt,,,,,,,
1,whole milk,pastry,salty snack,,,,,,,,
2,canned beer,misc. beverages,,,,,,,,,
3,sausage,hygiene articles,,,,,,,,,
4,soda,pickled vegetables,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
14958,tropical fruit,berries,other vegetables,yogurt,kitchen towels,napkins,,,,,
14959,bottled water,herbs,,,,,,,,,
14960,fruit/vegetable juice,onions,,,,,,,,,
14961,soda,root vegetables,semi-finished bread,,,,,,,,


In [14]:
records = []
for i in range(0, split_data.shape[0]):
    records.append([str(split_data.values[i, j]).strip() for j in range(0, split_data.shape[1]) if str(split_data.values[i, j]).strip()])

In [15]:
# Apply Apriori algorithm
association_rules = apriori(records, min_support=0.00030, min_confidence=0.05)
association_results = list(association_rules)

print(f"There are {len(association_results)} Relation derived.")

There are 1198 Relation derived.


In [16]:
def inspect(results):
    lhs = []
    rhs = []
    supports = []
    confidences = []
    lifts = []
    for result in results:
        for rule in result.ordered_statistics:
            lhs.append(', '.join(rule.items_base))
            rhs.append(', '.join(rule.items_add))
            supports.append(result.support)
            confidences.append(rule.confidence)
            lifts.append(rule.lift)
    return list(zip(lhs, rhs, supports, confidences, lifts))

In [17]:
# Create a DataFrame with the results
resultsDataFrame = pd.DataFrame(inspect(association_results), columns=["Left hand side", "Right hand side", "Support", "Confidence", "Lift"])

In [18]:
# Print first 10 rows with the highest Lift
resultsDataFrame.nlargest(n=10, columns="Lift")

,Left hand side,Right hand side,Support,Confidence,Lift
797,soups,seasonal products,0.000334,0.104167,14.704206
1215,"curd, fruit/vegetable juice",sausage,0.000334,0.500000,8.285161
1749,"other vegetables, pastry","soda, whole milk",0.000334,0.090909,7.817659
1029,"brown bread, frozen vegetables",canned beer,0.000334,0.357143,7.612434
1030,"canned beer, frozen vegetables",brown bread,0.000334,0.250000,6.644316
1118,"soda, yogurt",chewing gum,0.000401,0.068966,5.732950
1754,"soda, other vegetables, whole milk",pastry,0.000334,0.294118,5.685895
1066,"yogurt, butter milk",canned beer,0.000334,0.263158,5.609162
1756,"sausage, rolls/buns","whole milk, yogurt",0.000334,0.062500,5.599925
1750,"soda, pastry","whole milk, other vegetables",0.000334,0.081967,5.524664
